In [8]:
import os
import fnmatch

import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import operator

from textblob import TextBlob

from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import GridSearchCV

import pickle

In [9]:
path = 'op_spam_train/'
label = []
configfiles = [os.path.join(subdir,f)
for subdir, dirs, files in os.walk(path)
    for f in fnmatch.filter(files, '*.txt')]
len(configfiles)

1600

In [10]:
configfiles[1]

'op_spam_train/negative_polarity\\deceptive_from_MTurk\\fold1\\d_hilton_10.txt'

In [11]:
for f in configfiles:
    c = re.search('(trut|deceptiv)\w',f)
    label.append(c.group())
    
labels = pd.DataFrame(label, columns = ['Labels'])
labels.head(5)

Labels
0  deceptive
1  deceptive
2  deceptive
3  deceptive
4  deceptive

In [12]:
review = []
directory =os.path.join("op_spam_train/")
for subdir,dirs ,files in os.walk(directory):
   # print (subdir)
    for file in files:
        if fnmatch.filter(files, '*.txt'):
            f=open(os.path.join(subdir, file),'r')
            a = f.read()
            review.append(a)
reviews = pd.DataFrame(review, columns = ['HotelReviews'])
reviews.head(5)

HotelReviews
0  We stayed at the Schicago Hilton for 4 days an...
1  Hotel is located 1/2 mile from the train stati...
2  I made my reservation at the Hilton Chicago be...
3  When most people think Hilton, they think luxu...
4  My husband and I recently stayed stayed at the...

In [13]:
result = pd.merge(reviews, labels,right_index=True,left_index = True)

result['HotelReviews'] = result['HotelReviews'].map(lambda x: x.lower())
result.head()

HotelReviews     Labels
0  we stayed at the schicago hilton for 4 days an...  deceptive
1  hotel is located 1/2 mile from the train stati...  deceptive
2  i made my reservation at the hilton chicago be...  deceptive
3  when most people think hilton, they think luxu...  deceptive
4  my husband and i recently stayed stayed at the...  deceptive

In [14]:
html=result.to_html()
file=open("dataset.html","w")
file.write(html)
file.close()

In [15]:
result.tail()

HotelReviews Labels
1595  beautiful place with european charm. no compla...  truth
1596  spent three nights at the hotel for a girls we...  truth
1597  it has been a couple of years since i stayed h...  truth
1598  stayed here october 31 through november 5 for ...  truth
1599  we just returned from a girls shopping /sight-...  truth

In [16]:
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
result['review_without_stopwords'] = result['HotelReviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
result.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SHRUTHI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


HotelReviews     Labels  \
0  we stayed at the schicago hilton for 4 days an...  deceptive   
1  hotel is located 1/2 mile from the train stati...  deceptive   
2  i made my reservation at the hilton chicago be...  deceptive   
3  when most people think hilton, they think luxu...  deceptive   
4  my husband and i recently stayed stayed at the...  deceptive   

                            review_without_stopwords  
0  stayed schicago hilton 4 days 3 nights confere...  
1  hotel located 1/2 mile train station quite hik...  
2  made reservation hilton chicago believing goin...  
3  people think hilton, think luxury. know did. w...  
4  husband recently stayed stayed hilton chicago ...

In [17]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def pos(review_without_stopwords):
    return TextBlob(review_without_stopwords).tags
os = result.review_without_stopwords.apply(pos)
os1 = pd.DataFrame(os)
os1.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SHRUTHI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\SHRUTHI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


review_without_stopwords
0  [(stayed, VBN), (schicago, JJ), (hilton, NN), ...
1  [(hotel, NN), (located, VBD), (1/2, CD), (mile...
2  [(made, VBN), (reservation, NN), (hilton, NN),...
3  [(people, NNS), (think, VBP), (hilton, NN), (t...
4  [(husband, NN), (recently, RB), (stayed, VBD),...

In [18]:
os1['pos'] = os1['review_without_stopwords'].map(lambda x:" ".join(["/".join(x) for x in x ]) )

result = result = pd.merge(result, os1,right_index=True,left_index = True)
result.head()

HotelReviews     Labels  \
0  we stayed at the schicago hilton for 4 days an...  deceptive   
1  hotel is located 1/2 mile from the train stati...  deceptive   
2  i made my reservation at the hilton chicago be...  deceptive   
3  when most people think hilton, they think luxu...  deceptive   
4  my husband and i recently stayed stayed at the...  deceptive   

                          review_without_stopwords_x  \
0  stayed schicago hilton 4 days 3 nights confere...   
1  hotel located 1/2 mile train station quite hik...   
2  made reservation hilton chicago believing goin...   
3  people think hilton, think luxury. know did. w...   
4  husband recently stayed stayed hilton chicago ...   

                          review_without_stopwords_y  \
0  [(stayed, VBN), (schicago, JJ), (hilton, NN), ...   
1  [(hotel, NN), (located, VBD), (1/2, CD), (mile...   
2  [(made, VBN), (reservation, NN), (hilton, NN),...   
3  [(people, NNS), (think, VBP), (hilton, NN), (t...   
4  [(husband, NN), (recently, RB), (stayed, VBD),...   

                                                 pos  
0  stayed/VBN schicago/JJ hilton/NN 4/CD days/NNS...  
1  hotel/NN located/VBD 1/2/CD mile/NN train/NN s...  
2  made/VBN reservation/NN hilton/NN chicago/NN b...  
3  people/NNS think/VBP hilton/NN think/VBP luxur...  
4  husband/NN recently/RB stayed/VBD stayed/JJ hi...

In [19]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=13)

tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True, sublinear_tf=False)

X_train_tf = tf_vect.fit_transform(review_train)
X_test_tf = tf_vect.transform(review_test)

print(X_train_tf)

  (0, 8332)	0.08687070889269397
  (0, 3859)	0.036039941259879714
  (0, 4495)	0.07797526198274475
  (0, 5112)	0.051288921252897114
  (0, 5039)	0.0373920767077682
  (0, 5980)	0.06499296550045543
  (0, 2090)	0.1633344120464356
  (0, 3831)	0.0816672060232178
  (0, 7999)	0.06830179401185983
  (0, 3351)	0.06059181245207966
  (0, 7324)	0.03692535555959791
  (0, 2748)	0.06830179401185983
  (0, 5150)	0.03116657665063691
  (0, 4645)	0.09576615580264317
  (0, 2337)	0.061898032226145934
  (0, 5601)	0.04199856881108083
  (0, 2052)	0.08400701461580233
  (0, 3070)	0.090562652933167
  (0, 671)	0.0423934743429479
  (0, 3412)	0.05277543926001701
  (0, 8297)	0.06081326330083984
  (0, 4714)	0.053469306464367025
  (0, 6301)	0.14348907531280175
  (0, 8102)	0.0816672060232178
  (0, 500)	0.032910134547960046
  :	:
  (1279, 7322)	0.02385638041332856
  (1279, 2587)	0.06008045563428028
  (1279, 5148)	0.0407173572458275
  (1279, 3533)	0.03086383171864332
  (1279, 6782)	0.030233790661457965
  (1279, 2102)	0.054436

In [32]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_

svc_param_selection(X_train_tf,label_train,5)

{'C': 10, 'gamma': 0.001}

In [33]:
clf = svm.SVC(C=10,gamma=0.001,kernel='linear')
clf.fit(X_train_tf,label_train)
pred = clf.predict(X_test_tf)

In [34]:
with open('vectorizer.pickle', 'wb') as fin:
    pickle.dump(tf_vect, fin)
with open('mlmodel.pickle','wb') as f:
    pickle.dump(clf,f)

In [35]:
pkl = open('mlmodel.pickle', 'rb')
clf = pickle.load(pkl)   
vec = open('vectorizer.pickle', 'rb')
tf_vect = pickle.load(vec)

In [36]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [37]:
print(metrics.accuracy_score(label_test, pred))

0.49375


In [38]:
print (confusion_matrix(label_test, pred))

[[84 84]
 [78 74]]


In [39]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.52      0.50      0.51       168
       truth       0.47      0.49      0.48       152

    accuracy                           0.49       320
   macro avg       0.49      0.49      0.49       320
weighted avg       0.49      0.49      0.49       320



In [40]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=1)
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)
print(metrics.accuracy_score(label_test, pred))
print (confusion_matrix(label_test, pred))
print (classification_report(label_test, pred))


0.51875
[[77 74]
 [80 89]]
              precision    recall  f1-score   support

   deceptive       0.49      0.51      0.50       151
       truth       0.55      0.53      0.54       169

    accuracy                           0.52       320
   macro avg       0.52      0.52      0.52       320
weighted avg       0.52      0.52      0.52       320



In [41]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=10)
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)
print(metrics.accuracy_score(label_test, pred))
print (confusion_matrix(label_test, pred))
print (classification_report(label_test, pred))

0.51875
[[88 72]
 [82 78]]
              precision    recall  f1-score   support

   deceptive       0.52      0.55      0.53       160
       truth       0.52      0.49      0.50       160

    accuracy                           0.52       320
   macro avg       0.52      0.52      0.52       320
weighted avg       0.52      0.52      0.52       320



In [42]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=42)
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)
print(metrics.accuracy_score(label_test, pred))
print (confusion_matrix(label_test, pred))
print (classification_report(label_test, pred))

0.49375
[[84 84]
 [78 74]]
              precision    recall  f1-score   support

   deceptive       0.52      0.50      0.51       168
       truth       0.47      0.49      0.48       152

    accuracy                           0.49       320
   macro avg       0.49      0.49      0.49       320
weighted avg       0.49      0.49      0.49       320



In [46]:
result['Labels'].value_counts()

truth        800
deceptive    800
Name: Labels, dtype: int64

In [47]:
X=result.drop(columns='Labels',axis=1)
Y=result['Labels']

In [49]:
cv_score_svc = cross_val_score(SVC(kernel='linear'), X,Y,cv=5)
print(cv_score_svc)
mean_accuracy_svc = sum(cv_score_svc)/len(cv_score_svc)
mean_accuracy_svc = mean_accuracy_svc*100
mean_accuracy_svc=round(mean_accuracy_svc, 2)
print(mean_accuracy_svc)

[nan nan nan nan nan]
nan


C:\Users\SHRUTHI\anaconda3\envs\project\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\SHRUTHI\anaconda3\envs\project\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SHRUTHI\anaconda3\envs\project\lib\site-packages\sklearn\svm\_base.py", line 171, in fit
    accept_large_sparse=False)
  File "C:\Users\SHRUTHI\anaconda3\envs\project\lib\site-packages\sklearn\base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\SHRUTHI\anaconda3\envs\project\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\SHRUTHI\anaconda3\envs\project\lib\site-packages\sklearn\utils\validation.py", line 878, in c

In [ ]:
def test_string(s):
    X_test_tf = tf_vect.transform([s])
    y_predict = clf.predict(X_test_tf)
    return y_predict

In [ ]:
test_string("The hotel was bad.The room had a 27-inch Samsung led tv, a microwave.The room had a double bed")

In [ ]:
test_string("My husband and I recently stayed stayed at the Hilton Chicago and it was a horrible experience. When we arrived, the front desk clerk was on the phone on what sounded like a personal call and didn't even acknowledge us for at least five minutes. When we got to our room I went to lay on the bed and noticed a black hair on the pillow. I'm am very paranoid about hotels being dirty, so I pulled back the sheets and moved all the pillows around and noticed several more hairs. I called for some new sheets which they said they would send right up but it actually took nearly 30 minutes. The second night we were there we decided to try the Irish pub in the hotel called Kitty O'Sheas because my husband loves pub food. We had the shepherd's pie which was awful. It was watery and had no flavor and it was not cheap! Our server seemed annoyed when we complained and did not offer it to be taken off the check. While we had a great trip to Chicago, the Hilton was definitely NOT a highlight of our trip. Overall, we had rude service, a dirty room and bad food. Spend your money elsewhere")

In [ ]:
test_string("Not tasty and the texture was just nasty.")

In [ ]:
fig = plt.figure()
count_Class=pd.value_counts(result["Labels"], sort= True) 
count_Class.plot(kind= 'bar', color= ["red", "green"]) 
plt.title('Truthful and Deceptive data distribution')
plt.show() 
save_results_to = '/Users/Srujana/Desktop/project/Fake Restarent Review Classification/static/'
fig.savefig(save_results_to + 'graph.png')
